In [159]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup 
%matplotlib inline
pd.set_option("display.max_columns", 2000)

In [3]:
df_full=pd.read_csv('data/Seasons_Stats.csv')

In [227]:
df_full.sample(20)

,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,blanl,OWS,DWS,WS,WS/48,blank2,OBPM,DBPM,BPM,VORP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
1429,1959.0,Dave Piontek,PF,24.0,CIN,72.0,NaN,1674.0,13.4,0.420,NaN,0.279,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.1,0.6,0.5,0.014,NaN,NaN,NaN,NaN,NaN,305.0,813.0,0.375,NaN,NaN,NaN,305.0,813.0,0.375,0.375,156.0,227.0,0.687,NaN,NaN,385.0,124.0,NaN,NaN,NaN,162.0,766.0
23104,2015.0,Jorge Gutierrez,PG,26.0,TOT,20.0,1.0,175.0,11.0,0.567,0.073,0.317,3.3,12.9,8.1,19.6,1.8,0.0,20.4,15.2,NaN,0.1,0.2,0.3,0.088,NaN,-2.5,-0.6,-3.1,0.0,22.0,41.0,0.537,0.0,3.0,0.000,22.0,38.0,0.579,0.537,9.0,13.0,0.692,5.0,20.0,25.0,22.0,6.0,0.0,12.0,25.0,53.0
6681,1982.0,George McGinnis,PF,31.0,IND,76.0,4.0,1341.0,11.3,0.395,0.008,0.421,7.4,25.2,16.1,20.7,3.5,1.2,22.6,18.0,NaN,-2.5,2.9,0.4,0.015,NaN,-4.3,4.3,-0.1,0.7,141.0,378.0,0.373,0.0,3.0,0.000,141.0,375.0,0.376,0.373,72.0,159.0,0.453,93.0,305.0,398.0,204.0,96.0,28.0,131.0,198.0,354.0
16237,2003.0,Fred Jones,SG,23.0,IND,19.0,1.0,115.0,5.5,0.446,0.292,0.167,4.0,4.8,4.4,6.7,2.7,0.6,18.9,12.5,NaN,-0.1,0.1,0.0,0.019,NaN,-4.0,0.3,-3.7,0.0,9.0,24.0,0.375,2.0,7.0,0.286,7.0,17.0,0.412,0.417,3.0,4.0,0.750,4.0,5.0,9.0,5.0,6.0,1.0,6.0,13.0,23.0
5773,1980.0,Don Buse,PG,29.0,PHO,81.0,NaN,2499.0,10.7,0.485,0.134,0.217,3.1,7.0,5.1,16.0,2.4,0.2,12.4,11.7,NaN,1.7,2.9,4.6,0.088,NaN,-0.7,1.1,0.4,1.5,261.0,589.0,0.443,19.0,79.0,0.241,242.0,510.0,0.475,0.459,85.0,128.0,0.664,70.0,163.0,233.0,320.0,132.0,10.0,91.0,111.0,626.0
13062,1997.0,Grant Hill,SF,24.0,DET,80.0,80.0,3147.0,25.5,0.556,0.026,0.503,5.0,23.1,14.2,36.0,2.6,1.3,14.4,28.3,NaN,8.7,5.9,14.6,0.223,NaN,4.3,3.7,8.0,7.9,625.0,1259.0,0.496,10.0,33.0,0.303,615.0,1226.0,0.502,0.500,450.0,633.0,0.711,123.0,598.0,721.0,583.0,144.0,48.0,259.0,186.0,1710.0
18036,2006.0,Michael Olowokandi,C,30.0,TOT,48.0,24.0,919.0,8.3,0.456,0.000,0.196,6.7,21.7,14.4,4.1,1.3,2.7,17.4,16.2,NaN,-1.1,1.4,0.3,0.014,NaN,-6.0,0.8,-5.2,-0.7,107.0,240.0,0.446,0.0,0.0,NaN,107.0,240.0,0.446,0.446,24.0,47.0,0.511,50.0,170.0,220.0,23.0,22.0,33.0,55.0,122.0,238.0
9024,1988.0,Isiah Thomas*,PG,26.0,DET,81.0,81.0,2927.0,18.2,0.521,0.072,0.294,2.5,8.0,5.3,35.0,2.4,0.3,15.3,25.4,NaN,3.8,3.7,7.6,0.124,NaN,2.4,0.1,2.5,3.3,621.0,1341.0,0.463,30.0,97.0,0.309,591.0,1244.0,0.475,0.474,305.0,394.0,0.774,64.0,214.0,278.0,678.0,141.0,17.0,273.0,217.0,1577.0
3630,1973.0,Bob Dandridge,SF,25.0,MIL,73.0,NaN,2852.0,16.2,0.504,NaN,0.186,NaN,NaN,10.2,10.0,NaN,NaN,NaN,NaN,NaN,3.8,4.3,8.1,0.136,NaN,NaN,NaN,NaN,NaN,638.0,1353.0,0.472,NaN,NaN,NaN,638.0,1353.0,0.472,0.472,198.0,251.0,0.789,NaN,NaN,600.0,207.0,NaN,NaN,NaN,279.0,1474.0
12284,1995.0,David Wingate,SG,31.0,CHH,52.0,9.0,515.0,8.1,0.460,0.180,0.197,2.6,10.8,6.9,16.2,1.9,0.8,16.9,14.3,NaN,-0.2,0.6,0.4,0.037,NaN,-4.1,0.4,-3.8,-0.2,50.0,122.0,0.410,4.0,22.0,0.182,46.0,100.0,0.460,0.426,18.0,24.0,0.750,11.0,49.0,60.0,56.0,19.0,6.0,27.0,60.0,122.0


In [4]:
df_full.drop(columns='Unnamed: 0', inplace=True)

In [5]:
# Year - Season
# Player - name
# Pos - Position
# Age - Age
# Tm - Team
# G - Games
# GS - Games Started
# MP - Minutes Played
# PER - Player Efficiency Rating
# TS% - True Shooting %
# 3PAr - 3-Point Attempt Rate
# FTr - Free Throw Rate
# ORB% - Offensive Rebound Percentage
# DRB% - Defensive Rebound Percentage
# TRB% - Total Rebound Percentage
# AST% - Assist Percentage
# STL% - Steal Percentage
# BLK% - Block Percentage
# TOV% - Turnover Percentage
# USG% - Usage Percentage
# blanlempty
# OWS - Offensive Win Shares
# DWS - Defensive Win Shares
# WS - Win Shares
# WS/48Win Shares Per 48 Minutes
# blank2
# OBPM - Offensive Box Plus/Minus
# DBPM - Defensive Box Plus/Minus
# BPM - Box Plus/Minus
# VORPValue Over Replacement
# FG - Field Goals
# FGA - Field Goal Attempts
# FG% - Field Goal Percentage
# 3P - 3-Point Field Goals
# 3PA - 3-Point Field Goal Attempts
# 3P% - 3-Point Field Goal Percentage
# 2P - 2-Point Field Goals
# 2PA - 2-Point Field Goal Attempts
# 2P% - 2-Point Field Goal Percentage
# eFG% - Effective Field Goal Percentage
# FT - Free Throws
# FTA - Free Throw Attempts
# FT% - Free Throw Percentage
# ORB - Offensive Rebounds
# DRB - Defensive Rebounds
# TRB - Total Rebounds
# AST - Assists
# STL - Steals
# BLK - Blocks
# TOV - Turnovers
# PF - Personal Fouls
# PTS - Points

# https://www.basketball-reference.com/about/glossary.html for additional data

# use the past 12 years --> 2005 - 2017


In [6]:
df_full.head()

,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1950.0,Curly Armstrong,G-F,31.0,FTW,63.0,NaN,NaN,NaN,0.368,...,0.705,NaN,NaN,NaN,176.0,NaN,NaN,NaN,217.0,458.0
1,1950.0,Cliff Barker,SG,29.0,INO,49.0,NaN,NaN,NaN,0.435,...,0.708,NaN,NaN,NaN,109.0,NaN,NaN,NaN,99.0,279.0
2,1950.0,Leo Barnhorst,SF,25.0,CHS,67.0,NaN,NaN,NaN,0.394,...,0.698,NaN,NaN,NaN,140.0,NaN,NaN,NaN,192.0,438.0
3,1950.0,Ed Bartels,F,24.0,TOT,15.0,NaN,NaN,NaN,0.312,...,0.559,NaN,NaN,NaN,20.0,NaN,NaN,NaN,29.0,63.0
4,1950.0,Ed Bartels,F,24.0,DNN,13.0,NaN,NaN,NaN,0.308,...,0.548,NaN,NaN,NaN,20.0,NaN,NaN,NaN,27.0,59.0


In [7]:
df=df_full[df_full['Year']>=2005]

In [8]:
df.shape

(7603, 52)

In [9]:
df_full.shape

(24691, 52)

In [10]:
# df_coach=pd.read_csv('data/NBA_head_coaches.csv.xls')

In [11]:
# df_coach.info()

In [12]:
# splitting the teams into a list of years

In [13]:
# df_coach['Teams']=df_coach['Teams'].str.split(',')

In [14]:
# df.head()

In [15]:
# df_coach[df_coach['Name']=='Rick Adelman']

In [16]:
df['Year']=df['Year'].astype(int)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [17]:
#Scarping for coaches

In [18]:
 req = requests.get('https://www.basketball-reference.com/coaches/NBA_stats.html')

In [19]:
content=req.content

In [20]:
soup=BeautifulSoup(content) 

In [21]:
table=soup.find('table');

In [22]:
table.find_all('a')[1].text

'Richie Adubato'

In [23]:
coach_name=[]
for element in table.find_all('a'):
    coach_name.append(element.text)
    

In [24]:
table.find('a').attrs['href']

'/coaches/adelmri01c.html'

In [25]:
table.find_all('a')[0].attrs['href']

'/coaches/adelmri01c.html'

In [31]:
tables=table.find_all('a')

In [27]:
#scarping all of the coach tables

In [35]:
coach_list=[]
for element in tables:
    loc='https://www.basketball-reference.com/' + element.attrs['href']
    raw_coach_tables=requests.get(loc)
    raw_coach_table_content=raw_coach_tables.content
    soup2=BeautifulSoup(raw_coach_table_content) 
    coach_table_content=soup2.find('table')
    coach_list.append(pd.read_html(str(coach_table_content)))
   

In [36]:
len(coach_list)

329

In [39]:
sorted(coach_name);

In [28]:
#cleaning and merging coach table:

In [74]:

for idx,element in enumerate(coach_list):
#     element[0]=element[0][element[0]['Unnamed: 4_level_0']['G'].astype(str)!='Assistant Coach']
    element[0]['coach_name']=coach_name[idx]
    if idx==0:
        all_coach=element[0]
    else:
#         all_coach=all_coach.append(element[0])
        all_coach=all_coach.append(element[0])
            

3982

In [ ]:
# coach_columns={Team: 'Playoffs', 'Tm', Coach_Age: 'Regular Season','Age' , Season: 'Unnamed: 0_level_0','Season', ---,
#                Playoff_games'Unnamed: 11_level_0', 'G' ,---, playoff_wins: 'Unnamed: 12_level_0', 'W', ---,playoff_loses: 'Unnamed: 13_level_0', 'L'
#               ,---, playoff_W/L%: 'Unnamed: 14_level_0', 'W/L%', --- ,season_games:'Unnamed: 4_level_0', 'G', ---, season_wins: 'Unnamed: 5_level_0', 'W',
#               ---,season_loses: 'Unnamed: 6_level_0', 'L', ---, season_W/L% : 'Unnamed: 7_level_0', 'W/L%', --- ,'Unnamed: 8_level_0', 'W > .500',
#               ---, 'Unnamed: 9_level_0', 'Finish'}

In [76]:
len(all_coach.columns)

226

In [140]:
#write raw coach data to csv
all_coach.to_csv('data/coach_raw.csv')

In [105]:
# for element in range(226):
#     print(all_coach.columns[element])
#     print(all_coach[all_coach.columns[element][0]].head())

In [77]:
all_coach.head()

,Playoffs,Regular Season,Unnamed: 0_level_0,Unnamed: 100_level_0,Unnamed: 101_level_0,Unnamed: 102_level_0,Unnamed: 103_level_0,Unnamed: 104_level_0,Unnamed: 105_level_0,Unnamed: 106_level_0,...,Unnamed: 92_level_0,Unnamed: 93_level_0,Unnamed: 94_level_0,Unnamed: 95_level_0,Unnamed: 96_level_0,Unnamed: 97_level_0,Unnamed: 98_level_0,Unnamed: 99_level_0,Unnamed: 9_level_0,coach_name
,Tm,Age,Season,Unnamed: 100_level_1,Unnamed: 101_level_1,Unnamed: 102_level_1,Unnamed: 103_level_1,Unnamed: 104_level_1,Unnamed: 105_level_1,Unnamed: 106_level_1,...,Unnamed: 92_level_1,Unnamed: 93_level_1,Unnamed: 94_level_1,Unnamed: 95_level_1,Unnamed: 96_level_1,Unnamed: 97_level_1,Unnamed: 98_level_1,Unnamed: 99_level_1,Finish,
0,POR,37.0,1983-84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rick Adelman
1,POR,38.0,1984-85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rick Adelman
2,POR,39.0,1985-86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rick Adelman
3,POR,40.0,1986-87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rick Adelman
4,POR,41.0,1987-88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rick Adelman


In [105]:
df_coach=pd.DataFrame()

In [106]:
df_coach['Team']=all_coach['Playoffs']['Tm']

In [107]:
df_coach['coach_age']=all_coach['Regular Season']['Age']

In [108]:
df_coach['coach_name']=all_coach['coach_name']

In [109]:
df_coach['season']=all_coach['Unnamed: 0_level_0']['Season']

In [110]:
df_coach['playoff_games']=all_coach['Unnamed: 11_level_0']['G']

In [111]:
df_coach['playoff_wins']=all_coach['Unnamed: 12_level_0']['W']

In [112]:
df_coach['playoff_loses']=all_coach['Unnamed: 13_level_0']['L']

In [113]:
df_coach['playoff_W/L%']=all_coach['Unnamed: 14_level_0']['W/L%']

In [114]:
df_coach['season_games']=all_coach['Unnamed: 4_level_0']['G']

In [115]:
df_coach['season_wins']=all_coach['Unnamed: 5_level_0']['W']

In [116]:
df_coach['season_loses']=all_coach['Unnamed: 6_level_0']['L']

In [117]:
df_coach['season_W/L%']=all_coach['Unnamed: 7_level_0']['W/L%']

In [118]:
df_coach['season_position']=all_coach['Unnamed: 9_level_0']['Finish']

In [95]:
df_coach.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3982 entries, 0 to 5
Data columns (total 13 columns):
Team               3653 non-null object
coach_age          3180 non-null float64
coach_name         3982 non-null object
season             3982 non-null object
playoff_games      1094 non-null float64
playoff_wins       1094 non-null float64
playoff_loses      1094 non-null float64
playoff_W/L%       1094 non-null float64
season_games       3982 non-null object
season_wins        2149 non-null float64
season_loses       2149 non-null float64
season_W/L%        2149 non-null float64
season_position    1818 non-null float64
dtypes: float64(9), object(4)
memory usage: 435.5+ KB


In [96]:
#remove coaches before season 2005 and remove years = 'Career'

In [129]:
df_coach.dropna(axis=0,inplace=True, thresh=6) #dropped assistant coaches which have less than 6 fields filled

# Continue from here

In [141]:
# writing the coach data to csv:
df_coach.to_csv('data/coach_data_cleaned.csv')

In [102]:
# check to see all the data is in the table?

In [135]:
temp=[]
for element in coach_name:
#     if element not in df_coach['coach_name'].values:
    if element not in df_coach['coach_name'].values:
        temp.append(element)
        

In [136]:
sorted(temp)

[]

In [149]:
df_coach.sample(20)

,Team,coach_age,coach_name,season,playoff_games,playoff_wins,playoff_loses,playoff_W/L%,season_games,season_wins,season_loses,season_W/L%,season_position
26,NaN,NaN,Randy Wittman,Career,21.0,12.0,9.0,0.571,684,278.0,406.0,0.406,NaN
5,SFW,33.0,George Lee,1969-70,NaN,NaN,NaN,NaN,52,22.0,30.0,0.423,6.0
16,NaN,NaN,Frank Vogel,Career,61.0,31.0,30.0,0.508,595,304.0,291.0,0.511,NaN
18,IND,61.0,Jack Ramsay,1986-87,4.0,1.0,3.0,0.250,82,41.0,41.0,0.500,4.0
14,HOU,50.0,Bill Fitch,1984-85,5.0,2.0,3.0,0.400,82,48.0,34.0,0.585,2.0
9,POR,48.0,Maurice Cheeks,2004-05,NaN,NaN,NaN,NaN,55,22.0,33.0,0.400,4.0
18,SAS,60.0,Gregg Popovich,2008-09,5.0,1.0,4.0,0.200,82,54.0,28.0,0.659,1.0
14,GSW,49.0,Brian Winters,2001-02,NaN,NaN,NaN,NaN,59,13.0,46.0,0.220,7.0
19,PHI,54.0,Doug Moe,1992-93,NaN,NaN,NaN,NaN,56,19.0,37.0,0.339,6.0
3,PHI,42.0,Matt Guokas,1986-87,5.0,2.0,3.0,0.400,82,45.0,37.0,0.549,2.0


In [144]:
# removing "career" rows


In [152]:
df_coach=df_coach[df_coach['season']!='Career']

In [175]:
df_coach['season']=df_coach['season'].apply(lambda x: x[:2]+x[-2:]) #code for fixing ther dates

In [179]:
df_coach.head()

,Team,coach_age,coach_name,season,playoff_games,playoff_wins,playoff_loses,playoff_W/L%,season_games,season_wins,season_loses,season_W/L%,season_position
6,POR,42.0,Rick Adelman,1989,3.0,0.0,3.0,0.000,35,14.0,21.0,0.400,5.0
7,POR,43.0,Rick Adelman,1990,21.0,12.0,9.0,0.571,82,59.0,23.0,0.720,2.0
8,POR,44.0,Rick Adelman,1991,16.0,9.0,7.0,0.563,82,63.0,19.0,0.768,1.0
9,POR,45.0,Rick Adelman,1992,21.0,13.0,8.0,0.619,82,57.0,25.0,0.695,1.0
10,POR,46.0,Rick Adelman,1993,4.0,1.0,3.0,0.250,82,51.0,31.0,0.622,3.0


In [180]:
df.head()

,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,blanl,OWS,DWS,WS,WS/48,blank2,OBPM,DBPM,BPM,VORP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
17076,2005,Shareef Abdur-Rahim,PF,28.0,POR,54.0,49.0,1867.0,18.7,0.581,0.058,0.379,7.9,16.0,12.1,10.7,1.4,1.0,13.0,22.1,NaN,3.9,1.5,5.4,0.139,NaN,1.4,-0.5,0.8,1.3,337.0,670.0,0.503,15.0,39.0,0.385,322.0,631.0,0.510,0.514,220.0,254.0,0.866,123.0,269.0,392.0,111.0,49.0,26.0,117.0,150.0,909.0
17077,2005,Cory Alexander,SG,31.0,CHA,16.0,1.0,201.0,8.8,0.451,0.388,0.245,4.3,12.6,8.2,27.7,2.3,0.4,25.9,16.2,NaN,-0.2,0.2,0.0,0.002,NaN,-2.9,-0.5,-3.4,-0.1,16.0,49.0,0.327,8.0,19.0,0.421,8.0,30.0,0.267,0.408,9.0,12.0,0.750,8.0,21.0,29.0,37.0,9.0,1.0,19.0,29.0,49.0
17078,2005,Malik Allen,PF,26.0,TOT,36.0,1.0,519.0,14.2,0.512,0.000,0.158,9.0,13.3,11.0,5.7,0.9,3.2,8.7,18.1,NaN,0.6,0.5,1.1,0.100,NaN,-2.0,-0.3,-2.3,0.0,84.0,177.0,0.475,0.0,0.0,NaN,84.0,177.0,0.475,0.475,26.0,28.0,0.929,40.0,59.0,99.0,18.0,9.0,22.0,18.0,53.0,194.0
17079,2005,Malik Allen,PF,26.0,MIA,14.0,0.0,248.0,13.6,0.505,0.000,0.184,11.8,12.7,12.2,7.0,0.9,3.3,11.8,17.5,NaN,0.3,0.3,0.6,0.113,NaN,-1.7,0.6,-1.1,0.1,35.0,76.0,0.461,0.0,0.0,NaN,35.0,76.0,0.461,0.461,13.0,14.0,0.929,24.0,28.0,52.0,11.0,4.0,11.0,11.0,29.0,83.0
17080,2005,Malik Allen,PF,26.0,CHA,22.0,1.0,271.0,14.8,0.518,0.000,0.139,6.4,13.8,9.9,4.6,1.0,3.1,6.1,18.7,NaN,0.3,0.2,0.5,0.088,NaN,-2.4,-1.0,-3.4,-0.1,49.0,101.0,0.485,0.0,0.0,NaN,49.0,101.0,0.485,0.485,13.0,14.0,0.929,16.0,31.0,47.0,7.0,5.0,11.0,7.0,24.0,111.0


In [203]:
#taking only the coach name from the coah table and merging it to the player table
df_temp=pd.DataFrame()
df_temp['Year']=df_coach['season'].astype(int)
df_temp['Tm']=df_coach['Team']
df_temp['coach_name']=df_coach['coach_name']

In [204]:
df_temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1820 entries, 6 to 4
Data columns (total 3 columns):
Year          1820 non-null int64
Tm            1820 non-null object
coach_name    1820 non-null object
dtypes: int64(1), object(2)
memory usage: 56.9+ KB


In [206]:
df_temp.head()

,Year,Tm,coach_name
6,1989,POR,Rick Adelman
7,1990,POR,Rick Adelman
8,1991,POR,Rick Adelman
9,1992,POR,Rick Adelman
10,1993,POR,Rick Adelman


In [207]:
result=pd.merge(df,df_temp, how='left', on=['Year','Tm'])

In [234]:
result.sample(5) #this is the merged table

,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,blanl,OWS,DWS,WS,WS/48,blank2,OBPM,DBPM,BPM,VORP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,coach_name
6791,2015,Willie Green,SG,33.0,ORL,52.0,2.0,951.0,7.9,0.477,0.395,0.111,1.6,7.9,4.7,10.9,1.4,0.4,13.0,17.6,NaN,-0.4,0.4,-0.1,-0.003,NaN,-2.8,-2.1,-4.9,-0.7,118.0,306.0,0.386,42.0,121.0,0.347,76.0,185.0,0.411,0.454,28.0,34.0,0.824,14.0,65.0,79.0,68.0,26.0,5.0,48.0,81.0,306.0,James Borrego
5981,2014,Corey Brewer,SF,27.0,MIN,81.0,81.0,2609.0,12.7,0.550,0.261,0.281,2.6,6.2,4.3,7.9,2.9,0.9,10.4,16.5,NaN,2.4,2.7,5.2,0.095,NaN,0.4,1.3,1.7,2.4,388.0,807.0,0.481,59.0,211.0,0.280,329.0,596.0,0.552,0.517,163.0,227.0,0.718,63.0,144.0,207.0,135.0,151.0,30.0,105.0,210.0,998.0,Rick Adelman
6413,2014,Chris Singleton,SF,24.0,WAS,25.0,0.0,250.0,8.8,0.481,0.284,0.373,8.2,17.5,12.8,3.5,1.9,1.0,17.9,17.2,NaN,-0.2,0.3,0.2,0.033,NaN,-4.8,-0.9,-5.6,-0.2,25.0,67.0,0.373,7.0,19.0,0.368,18.0,48.0,0.375,0.425,18.0,25.0,0.720,18.0,37.0,55.0,6.0,9.0,3.0,17.0,25.0,75.0,Randy Wittman
2010,2008,Shannon Brown,SG,22.0,CHI,6.0,0.0,22.0,-11.8,0.255,0.067,0.400,5.0,5.3,5.1,0.0,2.3,7.1,22.1,45.5,NaN,-0.3,0.0,-0.3,-0.592,NaN,-24.3,-4.7,-29.0,-0.2,3.0,15.0,0.200,0.0,1.0,0.000,3.0,14.0,0.214,0.200,3.0,6.0,0.500,1.0,1.0,2.0,0.0,1.0,2.0,5.0,2.0,9.0,Jim Boylan
2697,2009,Jarron Collins,C,30.0,UTA,26.0,3.0,201.0,5.8,0.502,0.000,0.314,11.9,10.1,11.0,4.9,0.5,0.4,18.4,10.9,NaN,0.1,0.1,0.2,0.047,NaN,-2.9,-1.0,-3.9,-0.1,16.0,35.0,0.457,0.0,0.0,NaN,16.0,35.0,0.457,0.457,8.0,11.0,0.727,20.0,17.0,37.0,7.0,2.0,1.0,9.0,35.0,40.0,Jerry Sloan


# creating the player performance diff